## Multi-Party-Computation Scheme
- User $i = 1,2, 3$
- User $i$ randomly selects the number $r_j$ in R for $j \neq i$
- User $i$ devides the amount of his data to the following equations: $\sum_j m_{i,j}(t) = m_i(t)$
- The national dashboard will receive the data at time $t$ from user $i$ for all $i =1,2,3$ equal to $\sum_i\sum_j m_{i,j}(t) = \sum_i m_i(t)$ 

In [1]:
import random
import paho.mqtt.client as mqtt
from paho.mqtt import client as mqtt_client
import json
import time
import requests

## Give the data of manufacturing
- from t0 to t8

In [2]:
M1 = [400, 450, 500, 550, 500, 550, 400, 300, 250]
M2 = [300, 300, 400, 450, 500, 450, 400, 400, 300]
M3 = [200, 200, 500, 500, 300, 300, 300, 200, 200]

### Devide the data of user $i$ into 2 data for user $j$ and $k$

In [3]:
m_1_2 = []
m_1_3 = []
m_2_1 = []
m_2_3 = []
m_3_2 = []
m_3_1 = []

M_1_n = []
M_2_n = []
M_3_n = []

for i in range(0,9):
    
    r1 = random.randint(1,300)
    r2 = random.randint(1,300)
    r3 = random.randint(1,300)## select lower bound and upper bound
    
    m_1_2.append(r1)  
    m_1_3.append(M1[i] - r1)
    
    m_2_1.append(r2)  
    m_2_3.append(M2[i] - r2)
    
    m_3_1.append(r3)  
    m_3_2.append(M3[i] - r3)
    
    M_1_n.append(m_2_1[i] + m_3_1[i])
    M_2_n.append(m_1_2[i] + m_3_2[i])
    M_3_n.append(m_1_3[i] + m_2_3[i])



In [4]:
print("M1 will receive the data from M2 and M3 and will send them to the national dashboard as:",M_1_n )



M1 will receive the data from M2 and M3 and will send them to the national dashboard as: [39, 127, 181, 435, 277, 180, 247, 415, 128]


In [5]:
print("M2 will receive the data from M1 and M3 and will send them to the national dashboard as:",M_2_n)


M2 will receive the data from M1 and M3 and will send them to the national dashboard as: [240, 145, 453, 417, 154, 399, 288, -7, 410]


In [6]:
print("M3 will receive the data from M1 and M2 and will send them to the national dashboard as:",M_3_n)


M3 will receive the data from M1 and M2 and will send them to the national dashboard as: [621, 678, 766, 648, 869, 721, 565, 492, 212]


## Send Data to National Dashboard with NETPIE
- Define parameters to keep the data in NETPIE shadow
- M_1_n : the data that M1 sends to the NETPIE
- M_2_n : the data that M2 sends to the NETPIE
- M_3_n : the data that M3 sends to the NETPIE
- avg : the average value of all data at time t = $\sum_{i=1}^3 M_{i,n}(t)/3$  

In [7]:
## NETPIE device info
mqtt_client_name = "MPC"  # from NETPIE device
client_id = "d3297798-2b13-4179-91a0-4144e6189ed5" # client-ID from NETPIE device
username = "JeqTYLhaF9P5v5c5r34XSr5kD9mWLpjR" # Token from NETPIE device
password = "~E(CIP1nrvvFc2zp3q54f*SlWdHjrwz6" # secret from NETPIE device
broker = "broker.netpie.io" # MQTT broker
topic = "@shadow/data/update"
protocol = "mqtt"
PUBLISH_PERIOD = 2000  # milliseconds
port = 1883

In [8]:
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to NETPIE MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)
    # Set Connecting Client ID
    client = mqtt_client.Client(client_id)
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

def publish_shadow(client,c1,c2,c3):
    msg_dict = {"data": {'m1':c1,'m2':c2,'m3':c3,'avg':0}}
    msg = json.dumps(msg_dict)
    result = client.publish(topic, msg,qos=0, retain=False)
    
def subscribe(client_id,username):
    url = 'https://api.netpie.io/v2/device/shadow/data'
    basicAuthCredentials = (client_id, username)
    response = requests.get(url, auth=basicAuthCredentials)
    print(response.json())
    return response.json()

In [9]:
## Run NETPIE function
def run_NETPIE(c1,c2,c3):
    client = connect_mqtt()
    client.loop_start() # Start loop
    publish_shadow(client,c1,c2,c3) # publish data to @shadow/data/update
    time.sleep(2)
    client.loop_stop()
    result = subscribe(client_id,username)
    
    return result

## Test Send DATA to NETPIE

In [10]:
for i in range(0,9):
    print(i)
    print(M_1_n[i],M_2_n[i],M_3_n[i])
    m1 = M_1_n[i]
    m2 = M_2_n[i]
    m3 = M_3_n[i]
    result = run_NETPIE(m1,m2,m3)
    time.sleep(4)


0
39 240 621
Connected to NETPIE MQTT Broker!
{'deviceid': 'd3297798-2b13-4179-91a0-4144e6189ed5', 'data': {'avg': 300, 'm1': 39, 'm2': 240, 'm3': 621}, 'rev': 39, 'modified': 1651866663520}
1
127 145 678
Connected to NETPIE MQTT Broker!
{'deviceid': 'd3297798-2b13-4179-91a0-4144e6189ed5', 'data': {'avg': 316.6666666666667, 'm1': 127, 'm2': 145, 'm3': 678}, 'rev': 40, 'modified': 1651866670178}
2
181 453 766
Connected to NETPIE MQTT Broker!
{'deviceid': 'd3297798-2b13-4179-91a0-4144e6189ed5', 'data': {'avg': 466.6666666666667, 'm1': 181, 'm2': 453, 'm3': 766}, 'rev': 41, 'modified': 1651866676451}
3
435 417 648
Connected to NETPIE MQTT Broker!
{'deviceid': 'd3297798-2b13-4179-91a0-4144e6189ed5', 'data': {'avg': 500, 'm1': 435, 'm2': 417, 'm3': 648}, 'rev': 42, 'modified': 1651866682853}
4
277 154 869
Connected to NETPIE MQTT Broker!
{'deviceid': 'd3297798-2b13-4179-91a0-4144e6189ed5', 'data': {'avg': 433.3333333333333, 'm1': 277, 'm2': 154, 'm3': 869}, 'rev': 43, 'modified': 1651866689